In [93]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
gpt_turbo = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [33]:
import requests
url = 'https://public-api.meteofrance.fr/public/DPVigilance/v1/textesvigilance/encours'
headers = {
    'accept': '*/*',
    'apikey': os.environ['VIGILENCE_API_KEY']
}
response = requests.get(url, headers=headers)
data = response.json()

In [34]:
def extract_text_items(data):
    text_items = []
    infos = [
        region for region in data['product']['text_bloc_items'] 
        if len(region['bloc_items']) != 0 and region['bloc_id']=='BULLETIN_DEPARTEMENTAL'
    ]
    for info in infos:
        for bloc_item in info.get('bloc_items', []):
            for text_item in bloc_item.get('text_items', []):
                for subdivision in text_item.get('term_items', []):
                    for subdivision_text in subdivision.get('subdivision_text', []):
                        text_items.append("".join(subdivision_text.get('text', '')))
    return text_items

In [98]:
infos = list(set(extract_text_items(data)))

In [99]:
template = """
Fait un résumé concis des informations suivantes : {informations}
"""

In [100]:
prompt = PromptTemplate.from_template(template)


In [101]:
chain = (
    {'informations':RunnablePassthrough()} 
    | prompt 
    | gpt_turbo
)

In [104]:
infos

["Faits nouveaux : passage en vigilance orange vent violents sur le Finistère pour un début d'évènement à 16h ce lundi. Situation générale : Une dépression nommée PIERRICK se présente en entrée de Manche, proche du nord Finistère en ce début d'après-midi de lundi puis circule le long des côtes sud de l'Angleterre, engendrant des vents violents et un risque de vagues-submersion.",
 "La dépression PIERRICK concerne dans un premier temps le Finistère cet après-midi jusqu'en soirée. La dépression se décale ensuite vers l'est et donne encore de fortes rafales sur le département des Côtes d'Armor puis concerne le département de la Manche la nuit suivante. On attend 120 à 130 km/h sur les îles et à la côte du Finistère, autour de 110 km/h proche des littoraux et sur un quart nord-ouest du Finistère, 90 à ponctuellement plus de 100 km/h ailleurs sur le département. ",
 'Une incertitude persiste sur le positionnement exacte du creusement lié à PIERRICK. Ainsi on privilégie un scénario avec de v

In [102]:
answer = chain.invoke(infos)

In [103]:
print(answer)


La dépression nommée PIERRICK a entraîné le passage en vigilance orange vent violents sur le Finistère à partir de 16h ce lundi. La dépression a touché en premier lieu le Finistère avant de se déplacer vers l'est et de causer des vents forts sur les départements des Côtes d'Armor et de la Manche. Des rafales allant jusqu'à 130 km/h sont attendues sur les côtes et les îles du Finistère, avec un risque de submersion marine. Une incertitude demeure quant à la trajectoire exacte de la dépression, mais des vents violents sont également possibles sur le sud du département. La vigilance est maintenue pour les marées hautes de mardi matin et après-midi sur les départements du Finistère, des Côtes-d'Armor, de l'Ille-et-Vilaine, de la Manche, du Calvados, de la Seine-Maritime, de la Somme et du Pas-de-Calais en raison de fortes vagues et de risques de submersion. Les vents forts et les for


In [105]:
answer = chain.invoke(", ".join(infos))

In [106]:
print(answer)


La dépression PIERRICK a entraîné une vigilance orange pour des vents violents sur le Finistère à partir de 16h ce lundi. La dépression se déplace le long des côtes sud de l'Angleterre et devrait causer des rafales de vent allant jusqu'à 130 km/h sur les îles et les côtes du Finistère. Une incertitude persiste quant à la position exacte de la dépression, mais des vents violents sont également attendus sur le sud du département. Les départements de l'est de la Manche pourraient également voir leur vigilance "vagues-submersion" s'aggraver lors de la marée haute de mardi après-midi. La dépression se déplace ensuite en Mer du Nord, causant des vents forts et de fortes vagues sur la Manche. La conjonction de grandes marées et de fortes vagues nécessite une vigilance particulière pour les départements de la façade atlantique. La tempête, bien que tardive pour la saison, n'a pas de caractère exceptionnel mais peut causer des submersions marines sur les
